In [ ]:
# Database Environment Comparison Tool
from sqlalchemy import create_engine, text
import pandas as pd

SCHEMA = 'SCHEMA_NAME'

def connect_to_db(user, password, host, port, service_name):
    """
    Connect to Oracle database using SQLAlchemy.
    """
    # Fixed DSN connection string
    dsn = f'oracle+cx_oracle://{user}:{password}@{host}:{port}/?service_name={service_name}'
    engine = create_engine(dsn)
    return engine

def fetch_object_types(engine, is_uat):
    """
    Fetch distinct object types from database.
    """
    query = text(f"""
    SELECT DISTINCT OBJECT_TYPE
    FROM {'DBA_OBJECTS' if is_uat else 'ALL_OBJECTS'}
    WHERE OWNER = '{SCHEMA}'
    ORDER BY OBJECT_TYPE
    """)
    
    with engine.connect() as connection:
        result = connection.execute(query)
        object_types = [row[0] for row in result]
    return object_types

def fetch_object_names(engine, object_type, is_uat):
    """
    Fetch all object names of a specific type.
    """
    query = text(f"""
    SELECT OBJECT_NAME
    FROM {'DBA_OBJECTS' if is_uat else 'ALL_OBJECTS'}
    WHERE OWNER = '{SCHEMA}' AND OBJECT_TYPE = '{object_type}'
    ORDER BY OBJECT_NAME
    """)
    
    with engine.connect() as connection:
        result = connection.execute(query)
        objects = [row[0] for row in result]
    return objects

def compare_object_names(dev_objects, uat_objects, object_type):
    """
    Compare object names between DEV and UAT environments.
    """
    dev_set = set(dev_objects)
    uat_set = set(uat_objects)
    
    only_in_dev = dev_set - uat_set
    only_in_uat = uat_set - dev_set
    
    print(f"\n=== {object_type} Comparison ===")
    print(f"Number of {object_type}s in DEV: {len(dev_set)}")
    print(f"Number of {object_type}s in UAT: {len(uat_set)}")
    
    if not only_in_dev and not only_in_uat:
        print(f"✅ The {object_type} names in DEV and UAT are identical.")
    else:
        print(f"❌ The {object_type} names in DEV and UAT are different.")
        
        if only_in_dev:
            print(f"\n{object_type}s only in DEV ({len(only_in_dev)}):")
            for obj in sorted(only_in_dev):
                print(f"  - {obj}")
        
        if only_in_uat:
            print(f"\n{object_type}s only in UAT ({len(only_in_uat)}):")
            for obj in sorted(only_in_uat):
                print(f"- {obj}")
    
    print("-" * 50)
    return only_in_dev, only_in_uat

def generate_comparison_report(dev_engine, uat_engine, output_file="comparison_report.txt"):
    """
    Generate a comprehensive comparison report.
    """
    with open(output_file, 'w') as f:
        f.write("Database Environment Comparison Report\n")
        f.write("=" * 50 + "\n\n")
        
        # Fetch object types
        dev_object_types = fetch_object_types(dev_engine, is_uat=False)
        uat_object_types = fetch_object_types(uat_engine, is_uat=True)
        
        f.write(f"DEV Object Types: {dev_object_types}\n")
        f.write(f"UAT Object Types: {uat_object_types}\n\n")
        
        # Compare each object type
        common_object_types = set(dev_object_types) & set(uat_object_types)
        
        for object_type in sorted(common_object_types):
            dev_objects = fetch_object_names(dev_engine, object_type, is_uat=False)
            uat_objects = fetch_object_names(uat_engine, object_type, is_uat=True)
            
            only_in_dev, only_in_uat = compare_object_names(dev_objects, uat_objects, object_type)
            
            f.write(f"\n{object_type}:\n")
            f.write(f"DEV: {len(dev_objects)}, UAT: {len(uat_objects)}\n")
            if only_in_dev:
                f.write(f"Only in DEV: {', '.join(sorted(only_in_dev))}\n")
            if only_in_uat:
                f.write(f"Only in UAT: {', '.join(sorted(only_in_uat))}\n")
    
    print(f"📄 Report saved to: {output_file}")

In [ ]:
# Mock data for testing (add this to notebook)
def create_mock_comparison_test():
    """
    Creates a mock test to demonstrate the comparison functionality.
    """
    print("🧪 Running Mock Database Comparison Test")
    print("=" * 50)
    
    # Mock data
    dev_tables = ['USERS', 'PRODUCTS', 'ORDERS', 'CUSTOMERS']
    uat_tables = ['USERS', 'PRODUCTS', 'ORDERS']  # Missing CUSTOMERS
    
    dev_procedures = ['GET_USER', 'UPDATE_PRODUCT', 'PROCESS_ORDER']
    uat_procedures = ['GET_USER', 'UPDATE_PRODUCT', 'PROCESS_ORDER', 'NEW_PROCEDURE']  # Extra procedure
    
    # Test table comparison
    print("\n=== Testing TABLE Comparison ===")
    compare_mock_objects(dev_tables, uat_tables, 'TABLE')
    
    # Test procedure comparison
    print("\n=== Testing PROCEDURE Comparison ===")
    compare_mock_objects(dev_procedures, uat_procedures, 'PROCEDURE')
    
    print("\n✅ Mock test completed!")

def compare_mock_objects(dev_objects, uat_objects, object_type):
    """
    Mock version of compare_object_names for testing.
    """
    dev_set = set(dev_objects)
    uat_set = set(uat_objects)
    
    only_in_dev = dev_set - uat_set
    only_in_uat = uat_set - dev_set
    
    print(f"DEV {object_type}s: {len(dev_set)} ({', '.join(sorted(dev_set))})")
    print(f"UAT {object_type}s: {len(uat_set)} ({', '.join(sorted(uat_set))})")
    
    if not only_in_dev and not only_in_uat:
        print(f"✅ {object_type}s are identical in both environments")
    else:
        print(f"❌ {object_type}s differ between environments")
        if only_in_dev:
            print(f"Only in DEV: {', '.join(sorted(only_in_dev))}")
        if only_in_uat:
            print(f"Only in UAT: {', '.join(sorted(only_in_uat))}")

# Run the test
create_mock_comparison_test()

🧪 Running Mock Database Comparison Test

=== Testing TABLE Comparison ===
DEV TABLEs: 4 (CUSTOMERS, ORDERS, PRODUCTS, USERS)
UAT TABLEs: 3 (ORDERS, PRODUCTS, USERS)
❌ TABLEs differ between environments
  Only in DEV: CUSTOMERS

=== Testing PROCEDURE Comparison ===
DEV PROCEDUREs: 3 (GET_USER, PROCESS_ORDER, UPDATE_PRODUCT)
UAT PROCEDUREs: 4 (GET_USER, NEW_PROCEDURE, PROCESS_ORDER, UPDATE_PRODUCT)
❌ PROCEDUREs differ between environments
  Only in UAT: NEW_PROCEDURE

✅ Mock test completed!
